In [1]:
pip install pandas nltk fuzzywuzzy  rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.0 MB/s eta 0:00:00


In [2]:

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from fuzzywuzzy import process


/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# Load the Excel file
file_path = "/content/DA_Task_1.xlsx"

# Read the sheet names
xls = pd.ExcelFile(file_path)
xls.sheet_names

['complaints_data', 'taxonomy_sheet']

In [4]:
# Load the complaints data and taxonomy sheet
complaints_df = pd.read_excel(file_path, sheet_name="complaints_data")
taxonomy_df = pd.read_excel(file_path, sheet_name="taxonomy_sheet")

# Display the first few rows of each dataset
complaints_sample = complaints_df.head()
taxonomy_sample = taxonomy_df.head()

complaints_sample, taxonomy_sample

(    Primary Key Order Date Product Category  \
 0   SO0026296-1 2023-03-08           SPRAYS   
 1   SO0026385-1 2023-03-08           SPRAYS   
 2  SO0026385-11 2023-03-08           SPRAYS   
 3   SO0028352-1 2023-03-08           SPRAYS   
 4   SO0028770-1 2023-03-08           SPRAYS   
 
                                            Complaint  \
 0  VISIBLY NOTICE fasteners under cab on P clips ...   
 1                       Fuel door will not stay open   
 2   Compressor pressure line, braided steel, crushed   
 3                 Oil running from bottom of machine   
 4                   MISSING VECTOR & INTRIP UNLOCKS.   
 
                                                Cause  \
 0                            Not tighten at factory.   
 1     GAS STRUT NOT INSTALLED OR ANYWHERE ON MACHINE   
 2  Compressor pressure line, braided steel, crush...   
 3  OIL RETURN UNDER MACHINE SWIVEL FITTING LEFT L...   
 4  MISSING VECTOR & INTRIP UNLOCKS WERE NOT INSTA...   
 
                      

In [5]:
complaints_df.info()

taxonomy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Primary Key          20 non-null     object        
 1   Order Date           20 non-null     datetime64[ns]
 2   Product Category     20 non-null     object        
 3   Complaint            20 non-null     object        
 4   Cause                20 non-null     object        
 5   Correction           20 non-null     object        
 6   Root Cause           2 non-null      object        
 7   Symptom Condition 1  2 non-null      object        
 8   Symptom Component 1  2 non-null      object        
 9   Symptom Condition 2  1 non-null      object        
 10  Symptom Component 2  1 non-null      object        
 11  Symptom Condition 3  1 non-null      object        
 12  Symptom Component 3  1 non-null      object        
 13  Fix Condition 1      2 non-null      

In [6]:
# Select text-based columns for analysis
text_columns = ["Complaint", "Cause","Correction"]

# Fill missing values with an empty string
complaints_df[text_columns] = complaints_df[text_columns].fillna("")

In [7]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\W+', ' ', text)
    text = text.strip()
    return text

# Apply cleaning
for col in text_columns:
    complaints_df[col] = complaints_df[col].apply(clean_text)

In [8]:
# Select text-based columns for analysis
taxonomy_columns = ["Root Cause", "Symptom Condition ","Symptom Component", "Fix Condition", "Fix Component"]

# Fill missing values with an empty string
taxonomy_df[taxonomy_columns] = taxonomy_df[taxonomy_columns].fillna(taxonomy_df.mode())

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine all text data into a single corpus
corpus = complaints_df["Complaint"].tolist() + complaints_df["Cause"].tolist() + complaints_df["Correction"].tolist()

# Apply TF-IDF
vectorizer = TfidfVectorizer(max_features=1000, stop_words="english")
tfidf_matrix = vectorizer.fit_transform(corpus)

# Extract top keywords
keywords = vectorizer.get_feature_names_out()
print("Extracted Keywords:", keywords)


Extracted Keywords: ['ac' 'adapter' 'air' 'andnot' 'arrived' 'associated' 'auto' 'autoboom'
 'backorder' 'began' 'block' 'blown' 'bolts' 'boom' 'boomrunning' 'booms'
 'bosse' 'bracket' 'bracketpn' 'brackets' 'braided' 'broke' 'broken'
 'bulk' 'bulkhead' 'cab' 'callstart' 'came' 'case' 'checked' 'cleaned'
 'cleared' 'clips' 'cnh' 'code' 'codes' 'components' 'compressor'
 'condenser' 'connect' 'connection' 'connections' 'connectors'
 'contaminating' 'continue' 'control' 'cooler' 'coolers' 'counter'
 'coupler' 'coupling' 'cradle' 'crushed' 'customer' 'cut' 'damaged'
 'dealer' 'def' 'deliverd' 'depict' 'diagnose' 'did' 'disassemble' 'does'
 'door' 'drain' 'drip' 'dripping' 'drive' 'drove' 'ducting' 'elb' 'elbow'
 'end' 'engine' 'error' 'est' 'etim' 'factory' 'failed' 'farm' 'fast'
 'fasteners' 'fault' 'faulty' 'field' 'filled' 'fitting' 'fittings'
 'fittingunit' 'fold' 'folding' 'followed' 'frame' 'free' 'fuel' 'fully'
 'gallons' 'gas' 'getting' 'glass' 'good' 'got' 'hammer' 'hand' 'harnes

In [10]:
taxonomy_df.columns = taxonomy_df.columns.str.strip()

In [11]:
from rapidfuzz import process
from fuzzywuzzy import process


In [12]:
def tag_text(text, category_list, threshold=50):
    if pd.isna(text):
        return "Unknown"
    text = str(text).strip()
    if not text:
        return "Unknown"

    if not category_list:
        return "Unknown"

    best_match, score = process.extractOne(text, category_list, scorer=process.fuzz.WRatio)
    return best_match if score > threshold else "Unknown"

def extract_conditions_from_text(text, symptom_condition_list, symptom_component_list,
                                  fix_condition_list, fix_component_list, root_cause_list):
    """Extract multiple conditions and components from text"""
    conditions = []
    components = []
    root_causes = []

    for condition in symptom_condition_list:
        if process.extractOne(condition, [text], scorer=process.fuzz.WRatio)[1] > 50:
            conditions.append(condition)
    for component in symptom_component_list:
        if process.extractOne(component, [text], scorer=process.fuzz.WRatio)[1] > 50:
            components.append(component)
    for condition in fix_condition_list:
        if process.extractOne(condition, [text], scorer=process.fuzz.WRatio)[1] > 50:
            conditions.append(condition)
    for component in fix_component_list:
        if process.extractOne(component, [text], scorer=process.fuzz.WRatio)[1] > 50:
            components.append(component)
    for root_cause in root_cause_list:
        if process.extractOne(root_cause, [text], scorer=process.fuzz.WRatio)[1] > 50:
            root_causes.append(root_cause)

    return conditions, components, root_causes

def process_complaints_data(complaints_df, taxonomy_df):
    taxonomy_df = taxonomy_df.applymap(lambda x: str(x).strip().lower() if isinstance(x, str) else x)
    symptom_condition_list = taxonomy_df["Symptom Condition"].dropna().unique().tolist()
    symptom_component_list = taxonomy_df["Symptom Component"].dropna().unique().tolist()
    fix_condition_list = taxonomy_df["Fix Condition"].dropna().unique().tolist()
    fix_component_list = taxonomy_df["Fix Component"].dropna().unique().tolist()
    root_cause_list = taxonomy_df["Root Cause"].dropna().unique().tolist()
    text_columns = ["Complaint", "Cause", "Correction"]

    for idx, row in complaints_df.iterrows():
        all_conditions = []
        all_components = []
        all_root_causes = []
        for col in text_columns:
            conditions, components, root_causes = extract_conditions_from_text(
                row[col],
                symptom_condition_list,
                symptom_component_list,
                fix_condition_list,
                fix_component_list,
                root_cause_list
            )
            all_conditions.extend(conditions)
            all_components.extend(components)
            all_root_causes.extend(root_causes)
        all_conditions = list(dict.fromkeys(all_conditions))
        all_components = list(dict.fromkeys(all_components))
        all_root_causes = list(dict.fromkeys(all_root_causes))
        for i in range(min(3, len(all_conditions))):
            complaints_df.at[idx, f"Symptom Condition {i+1}"] = all_conditions[i]
        for i in range(min(3, len(all_components))):
            complaints_df.at[idx, f"Symptom Component {i+1}"] = all_components[i]
        fix_conditions = [c for c in all_conditions if c in fix_condition_list]
        fix_components = [c for c in all_components if c in fix_component_list]

        for i in range(min(3, len(fix_conditions))):
            complaints_df.at[idx, f"Fix Condition {i+1}"] = fix_conditions[i]

        for i in range(min(3, len(fix_components))):
            complaints_df.at[idx, f"Fix Component {i+1}"] = fix_components[i]


        if all_root_causes:
            complaints_df.at[idx, "Root Cause"] = all_root_causes[0]
    columns_to_fill = [f"{prefix} {i+1}" for prefix in
                      ["Symptom Condition", "Symptom Component", "Fix Condition", "Fix Component"]
                      for i in range(3)]
    columns_to_fill.append("Root Cause")

    for col in columns_to_fill:
        complaints_df[col] = complaints_df[col].fillna("Unknown")

    return complaints_df

In [13]:
# Main execution
if 'complaints_df' in globals() and 'taxonomy_df' in globals():
    # Process the data
    tagged_complaints_df = process_complaints_data(complaints_df.copy(), taxonomy_df)

    # Save to Excel
    tagged_complaints_df.to_excel("Tagged_Data_Enhanced.xlsx", index=False)
    print("Data processing completed and saved to Tagged_Data_Enhanced.xlsx")
else:
    print("Error: complaints_df and taxonomy_df must be defined before processing.")

<ipython-input-12-050af133386d>:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  taxonomy_df = taxonomy_df.applymap(lambda x: str(x).strip().lower() if isinstance(x, str) else x)


Data processing completed and saved to Tagged_Data_Enhanced.xlsx


**Thank you for your patience and understanding...!**